# **Case Study: Generating Insights from E-Commerce Orders Data**

### **What's covered in this notebook?**
1. Reading JSON File
2. Understanding the JSON Structure
3. Creating Tables from JSON for Analysis
	- Normalizing JSON
	- Creating Delivery Table
	- Creating Payment Table
	- Creating Order Item Table
	- Creating Order Delivery History Table
4. Brainstorming
5. Top-Selling Categories & Products (By Quantity)
6. Calculate Total Payment After Discounts
	- Why use a LEFT Join in this case?
7. Most Popular Payment Methods
8. Extract Latest Status for Each Order
9. Compare All Order Statuses
10. Delivery Performance Analysis
	- Handling Missing Values
	- Calculating Delivery Status Breakdown
	- Calculating On-Time and Delayed Delivery Rate
	- Calculating Average Delivery Time
	- Calculating Carrier Performance

## **Reading JSON File**

In [1]:
import json

with open("ecommerce_data/orders.json", 'r') as file:
    data = json.load(file)

print(f"Data loaded successfully inside a variable with {type(data)} type and {len(data)} orders.")

Data loaded successfully inside a variable with <class 'list'> type and 1000000 orders.


In [2]:
json_sample = json.dumps(data[0:3], indent=4)

print(json_sample)

[
    {
        "order_id": "ORD1",
        "customer": {
            "id": 1639,
            "name": "Angela Griffin",
            "email": "gibbsedward@example.org",
            "address": {
                "street": "24026 Darlene Ranch",
                "city": "Angelashire",
                "country": "Saint Helena"
            }
        },
        "items": [
            {
                "product_id": 25,
                "name": "Mountain Bike",
                "category": "Sports",
                "price": 599.99,
                "quantity": 1
            },
            {
                "product_id": 23,
                "name": "Tennis Racket",
                "category": "Sports",
                "price": 89.99,
                "quantity": 2
            }
        ],
        "payment": {
            "method": "Credit Card",
            "transaction_id": "D005B042-A",
            "discount_applied": 2.48
        },
        "delivery": {
            "status": "Delivered",
       

## **Understanding the JSON Structure**  
Your JSON data contains the following key elements:

1. `order_id` (Unique for each order)
2. `customer` (Details about the customer)
   - `customer_id`
   - `name`
   - `email`
   - `address` (Street, City, Country)
3. `items` (List of products purchased in the order)
   - `product_id`
   - `name`
   - `category`
   - `price`
   - `quantity`
4. `payment` (Payment details)
   - `method`
   - `transaction_id`
   - `discount_applied`
5. `delivery` (Delivery details)
   - `status`
   - `tracking_id`
   - `shipping_company`
   - `expected_delivery_date`
6. `order_history` (List of order statuses with timestamps)


## **Creating Tables from JSON for Analysis**

---

To normalize the data, we can create the following **five tables**:

#### **Orders Table**
| order_id | customer_id |
|----------|------------|
| ORD12345 | 98765      |
| ORD67890 | 54321      |
| ORD24680 | 11223      |

> **Reason:** Orders should have a **one-to-one** relationship with customers. The `customer_id` acts as a **foreign key**.

---

#### **Customers Table**
| customer_id | name        | email                  | street       | city         | country |
|------------|------------|----------------------|-------------|-------------|---------|
| 98765      | John Doe   | johndoe@email.com    | 123 Main St | New York    | USA     |
| 54321      | Jane Smith | janesmith@email.com  | 456 Elm St  | Los Angeles | USA     |
| 11223      | Alice Johnson | alicejohnson@email.com | 789 Oak St  | Chicago  | USA     |

> **Reason:** Customer data should be stored separately for **data reusability**.

---

#### **Order Items Table**
| order_id | product_id | name                     | category      | price  | quantity |
|----------|------------|-------------------------|--------------|--------|----------|
| ORD12345 | 111        | Laptop                  | Electronics   | 1500.00 | 1        |
| ORD12345 | 222        | Wireless Mouse          | Accessories   | 50.00  | 2        |
| ORD67890 | 333        | Smartphone              | Electronics   | 999.99 | 1        |
| ORD24680 | 444        | Gaming Headset          | Accessories   | 199.99 | 1        |
| ORD24680 | 555        | Mechanical Keyboard     | Accessories   | 120.00 | 1        |

> **Reason:** Orders can have **multiple items**, so a **separate table** is needed.

---

#### **Payments Table**
| order_id | payment_method | payment_transaction_id | payment_discount_applied |
|----------|---------------|------------------------|-------------------------|
| ORD12345 | Credit Card   | TXN78910               | 10.00                   |
| ORD67890 | PayPal        | TXN65432               | 5.00                    |
| ORD24680 | Debit Card    | TXN98765               | 15.00                   |

> **Reason:** Payment details should be **stored separately** to maintain **transaction integrity**.

---

#### **Order History Table**
| order_id | status     | timestamp             |
|----------|-----------|-----------------------|
| ORD12345 | Processing | 2025-03-16T10:00:00  |
| ORD12345 | Shipped   | 2025-03-17T12:00:00  |
| ORD12345 | Delivered | 2025-03-19T15:00:00  |
| ORD67890 | Processing | 2025-03-16T14:00:00  |
| ORD67890 | Shipped   | 2025-03-18T10:00:00  |
| ORD24680 | Processing | 2025-03-17T09:30:00  |

> **Reason:** Orders go through multiple statuses, so we need a **history table**.

---

### **Normalizing JSON**

In [3]:
import pandas as pd

df = pd.json_normalize(data, sep="_")

df.head()

,order_id,items,order_history,customer_id,customer_name,customer_email,customer_address_street,customer_address_city,customer_address_country,payment_method,payment_transaction_id,payment_discount_applied,delivery_status,delivery_tracking_id,delivery_shipping_company,delivery_expected_delivery_date
0,ORD1,"[{'product_id': 25, 'name': 'Mountain Bike', '...","[{'status': 'Processing', 'timestamp': '2025-0...",1639,Angela Griffin,gibbsedward@example.org,24026 Darlene Ranch,Angelashire,Saint Helena,Credit Card,D005B042-A,2.48,Delivered,51D506FA-5,UPS,2025-03-14
1,ORD2,"[{'product_id': 24, 'name': 'Hydration Backpac...","[{'status': 'Processing', 'timestamp': '2025-0...",163,Sarah Moore,zshelton@example.org,33466 Kristin Meadow Suite 060,Lake Brittany,Nigeria,Credit Card,C1A2BCA7-1,6.12,Delivered,C188AB48-0,FedEx,2025-03-12
2,ORD3,"[{'product_id': 28, 'name': 'Noise-Canceling O...","[{'status': 'Processing', 'timestamp': '2025-0...",1814,Dwayne Hartman,daviddyer@example.org,32036 Rodney Creek,New Brandy,Sierra Leone,Credit Card,98F4114D-B,5.42,Processing,None,None,2025-03-18
3,ORD4,"[{'product_id': 10, 'name': 'Noise-Isolating E...","[{'status': 'Processing', 'timestamp': '2025-0...",4085,Michael Walters,michele19@example.com,09372 Collins Meadows,New Kimberly,Congo,Credit Card,269AE633-8,14.43,Shipped,B3D21590-A,USPS,2025-03-13
4,ORD5,"[{'product_id': 2, 'name': '4K Ultra HD Smart ...","[{'status': 'Processing', 'timestamp': '2025-0...",2908,Heather Jones,jshaw@example.net,479 Kimberly Ville Suite 888,Port Taylor,Grenada,Credit Card,2C6598E5-B,464.80,Delivered,649F931C-3,BlueDart,2025-03-16


In [4]:
df.columns

Index(['order_id', 'items', 'order_history', 'customer_id', 'customer_name',
       'customer_email', 'customer_address_street', 'customer_address_city',
       'customer_address_country', 'payment_method', 'payment_transaction_id',
       'payment_discount_applied', 'delivery_status', 'delivery_tracking_id',
       'delivery_shipping_company', 'delivery_expected_delivery_date'],
      dtype='object')

### **Creating Delivery Table**

In [5]:
# Extracting only the delivery-related fields
df_deliveries = df[["order_id", "delivery_status", "delivery_tracking_id", 
                  "delivery_shipping_company", "delivery_expected_delivery_date"]].copy()

df_deliveries.head()

,order_id,delivery_status,delivery_tracking_id,delivery_shipping_company,delivery_expected_delivery_date
0,ORD1,Delivered,51D506FA-5,UPS,2025-03-14
1,ORD2,Delivered,C188AB48-0,FedEx,2025-03-12
2,ORD3,Processing,None,None,2025-03-18
3,ORD4,Shipped,B3D21590-A,USPS,2025-03-13
4,ORD5,Delivered,649F931C-3,BlueDart,2025-03-16


### **Creating Payment Table**

In [6]:
# Extracting only the payment-related fields
df_payments = df[["order_id", "payment_method", "payment_transaction_id", 
                  "payment_discount_applied"]].copy()

df_payments.head()

,order_id,payment_method,payment_transaction_id,payment_discount_applied
0,ORD1,Credit Card,D005B042-A,2.48
1,ORD2,Credit Card,C1A2BCA7-1,6.12
2,ORD3,Credit Card,98F4114D-B,5.42
3,ORD4,Credit Card,269AE633-8,14.43
4,ORD5,Credit Card,2C6598E5-B,464.80


### **Creating Order Item Table**

In [9]:
df_order_items = pd.json_normalize(data, 
                                  sep="_", 
                                  record_path=["items"], 
                                  meta=["order_id"]
                                  )

df_order_items.head()

,product_id,name,category,price,quantity,order_id
0,25,Mountain Bike,Sports,599.99,1,ORD1
1,23,Tennis Racket,Sports,89.99,2,ORD1
2,24,Hydration Backpack for Runners,Sports,59.99,1,ORD2
3,21,Yoga Mat with Non-Slip Surface,Sports,39.99,2,ORD2
4,28,Noise-Canceling Office Headset,Accessories,89.99,1,ORD3


### **Creating Order Delivery History Table**

In [10]:
# Extract Order History in a Relational Format
df_order_history = pd.json_normalize(data, 
                                     sep="_",
                                     record_path=["order_history"], 
                                     meta=["order_id"])

df_order_history.head()

,status,timestamp,order_id
0,Processing,2025-03-10T22:35:56,ORD1
1,Shipped,2025-03-11T22:35:56,ORD1
2,Delivered,2025-03-13T22:35:56,ORD1
3,Processing,2025-03-09T18:15:56,ORD2
4,Shipped,2025-03-11T18:15:56,ORD2


## **Brainstorming**
When dealing with millions of orders, businesses must analyze vast amounts of data efficiently. Without structured insights, decision-making becomes reactive instead of proactive. Here’s why this level of analysis is critical for scaling operations, improving profitability, and enhancing customer experience. 

**Business Goal:** 
- Identify best-performing products & categories to optimize inventory and marketing.
- Understand customer payment preferences to optimize checkout experience.
- Ensure correct discount application and understand pricing trends.
- Provide real-time order tracking insights.
- Analyze order processing efficiency.
- Identify bottlenecks in shipping & improve logistics efficiency.

**Questions to Analyse:**
- What are the top 10 best-selling products by total quantity sold?
- What are the top 10 best-selling products by total revenue?
- What are the most used payment methods across all orders?
- Do certain payment methods have higher total revenue than others?
- Are certain payment methods associated with higher refund or failed transactions?
- What is the total amount paid for each order after discount?
- How much revenue was lost due to discounts?
- What is the latest recorded status for each order?
- How many orders are currently in each stage of the process (Processing, Shipped, Delivered, Cancelled)?
- Which shipping company has the best on-time delivery rate?
- etc...

## **Top-Selling Categories & Products (By Quantity)**

In [11]:
top_products = df_order_items.groupby("name")["quantity"].sum().sort_values(ascending=False)

print(top_products)

name
Hydration Backpack for Runners         485118
Tennis Racket                          403387
Wireless Noise-Canceling Headphones    403080
Bluetooth Portable Speaker             243451
4K Ultra HD Smart TV                   242775
Mountain Bike                          242755
Noise-Isolating Earbuds                242642
Winter Puffer Jacket                   241756
Yoga Mat with Non-Slip Surface         161800
Ergonomic Wireless Mouse               161512
Home Theater Speaker System            161442
Adjustable Standing Desk               160239
High-Powered Vacuum Cleaner             81466
Slim Fit Denim Jeans                    81364
Ceramic Dinnerware Set                  81349
Premium Memory Foam Pillow              81315
Men's Waterproof Jacket                 81176
Smartwatch with Heart Rate Monitor      81058
Stainless Steel Cookware Set            81000
USB-C Charging Dock                     80891
Cotton Crew Neck T-Shirt                80852
Adjustable Dumbbell Set      

In [12]:
top_categories = df_order_items.groupby("category")["quantity"].sum().sort_values(ascending=False)

print("Top Categories:\n", top_categories)

Top Categories:
 category
Sports         1373894
Electronics    1292973
Accessories     726570
Clothing        646019
Home            485369
Name: quantity, dtype: int64


## **Most Popular Payment Methods**

In [13]:
top_payment_methods = df_payments["payment_method"].value_counts()

print(top_payment_methods)

payment_method
Credit Card      499622
PayPal           300750
Debit Card       149589
Bank Transfer     50039
Name: count, dtype: int64


## **Calculate Total Payment After Discounts**

In [14]:
df_order_items["total_amount"] = df_order_items["price"] * df_order_items["quantity"]

df_order_items.head()

,product_id,name,category,price,quantity,order_id,total_amount
0,25,Mountain Bike,Sports,599.99,1,ORD1,599.99
1,23,Tennis Racket,Sports,89.99,2,ORD1,179.98
2,24,Hydration Backpack for Runners,Sports,59.99,1,ORD2,59.99
3,21,Yoga Mat with Non-Slip Surface,Sports,39.99,2,ORD2,79.98
4,28,Noise-Canceling Office Headset,Accessories,89.99,1,ORD3,89.99


In [17]:
# Aggregate total amount per order
df_total = df_order_items.groupby("order_id", as_index=False)["total_amount"].sum()

df_total.head()

,order_id,total_amount
0,ORD1,779.97
1,ORD10,59.99
2,ORD100,159.98
3,ORD1000,629.96
4,ORD10000,259.97


In [18]:
# # Merge with payment details
df_merged = df_total.merge(df_payments, on="order_id", how="left")

df_merged.head()

,order_id,total_amount,payment_method,payment_transaction_id,payment_discount_applied
0,ORD1,779.97,Credit Card,D005B042-A,2.48
1,ORD10,59.99,Credit Card,25A6367A-9,1.36
2,ORD100,159.98,Credit Card,B54E4367-7,1.01
3,ORD1000,629.96,Debit Card,673C4FBF-D,63.49
4,ORD10000,259.97,Credit Card,5A384D39-3,8.60


In [19]:
# Compute the final payment after discount
df_merged["total_payment_after_discount"] = df_merged["total_amount"] - df_merged["payment_discount_applied"]

# Display the result
df_merged.head()

,order_id,total_amount,payment_method,payment_transaction_id,payment_discount_applied,total_payment_after_discount
0,ORD1,779.97,Credit Card,D005B042-A,2.48,777.49
1,ORD10,59.99,Credit Card,25A6367A-9,1.36,58.63
2,ORD100,159.98,Credit Card,B54E4367-7,1.01,158.97
3,ORD1000,629.96,Debit Card,673C4FBF-D,63.49,566.47
4,ORD10000,259.97,Credit Card,5A384D39-3,8.60,251.37


In [20]:
df_merged.describe()

# NOTE: All three columns values are independent of each other

,total_amount,payment_discount_applied,total_payment_after_discount
count,1000000.000000,1000000.00000,1000000.000000
mean,1064.811247,79.92195,984.889297
std,942.870049,93.93399,874.165848
min,19.990000,0.00000,16.990000
25%,359.950000,16.83000,328.690000
50%,749.970000,45.11000,688.070000
75%,1559.970000,107.24000,1444.690000
max,8999.890000,1133.46000,8134.020000


In [21]:
print(f"Total Revenue After Discount: {df_merged['total_payment_after_discount'].sum()}")

Total Revenue After Discount: 984889296.7699994


### **Why use a LEFT Join in this case?**

We have two tables:
1. df_total (Left Table): Contains order_id and the total amount of all items in that order.
2. df_payments (Right Table): Contains order_id, payment details, and the discount applied.

**Our goal:** Compute the total payment after discount for each order.

**Why NOT Use an INNER JOIN?**
```python
df_merged = df_total.merge(df_payments, on="order_id", how="inner")
```
If we use the above it would only keep orders that exist in both tables. This means:

1. Orders present in both df_total and df_payments will be included.
2. If an order exists in df_total but has no payment record in df_payments, it will be excluded.

This is not desirable because we want to keep all order totals, even if no payment record exists.

**Why LEFT JOIN is the Best Choice?**  
1. We keep all orders from df_total (because they represent actual purchases).
2. If a payment record exists in df_payments, it gets merged in.
3. If a payment record does NOT exist, we still keep the order, but NaN appears in the payment_discount_applied column.

**What Happens to Missing Orders?**

Let’s assume ORD4 exists in df_payments but not in df_orders. A LEFT JOIN ensures that:

| order_id | total_amount | payment_discount_applied |
|----------|-------------|-------------------------|
| ORD1     | 64.98       | 1.93                    |
| ORD2     | 389.97      | 23.34                   |
| ORD3     | 489.95      | 184.62                  |
| ORD4     | NaN         | 20.87                   |  
| ORD5     | NaN         | 30.88                   |

Here, ORD4 and ORD5 appear with NaN in total_amount, meaning no order records exist for them, which we can handle later (e.g., replace NaN with 0).

## **Extract Latest Status for Each Order**

In [22]:
# Convert timestamp to datetime for sorting
df_order_history["timestamp"] = pd.to_datetime(df_order_history["timestamp"])

# Get latest status for each order
df_latest_status = df_order_history.sort_values(by=["order_id", "timestamp"]).groupby("order_id").last().reset_index()

df_latest_status.head()

,order_id,status,timestamp
0,ORD1,Delivered,2025-03-13 22:35:56
1,ORD10,Processing,2025-03-12 20:30:56
2,ORD100,Processing,2025-03-16 18:54:56
3,ORD1000,Delivered,2025-03-17 20:20:56
4,ORD10000,Processing,2025-03-09 18:23:58


## **Compare All Order Statuses**

In [23]:
# Assign a unique step number for each order status
df_order_history["step"] = df_order_history.groupby("order_id").cumcount() + 1

df_order_history.head()

,status,timestamp,order_id,step
0,Processing,2025-03-10 22:35:56,ORD1,1
1,Shipped,2025-03-11 22:35:56,ORD1,2
2,Delivered,2025-03-13 22:35:56,ORD1,3
3,Processing,2025-03-09 18:15:56,ORD2,1
4,Shipped,2025-03-11 18:15:56,ORD2,2


In [24]:
# Pivot the data to spread out the statuses
df_pivot = df_order_history.pivot(index="order_id", columns="step", values="status")

df_pivot.head()

step,1,2,3
order_id,,,
ORD1,Processing,Shipped,Delivered
ORD10,Processing,NaN,NaN
ORD100,Processing,NaN,NaN
ORD1000,Processing,Shipped,Delivered
ORD10000,Processing,NaN,NaN


## **Delivery Performance Analysis**

Delivery Performance Analysis evaluates the efficiency and reliability of order fulfillment by measuring key delivery metrics. It helps businesses assess their logistics performance, customer satisfaction, and operational bottlenecks.

**Key Metrics in Delivery Performance Analysis**
1. Delivery Status Breakdown – Number of orders in different stages (Delivered, In Transit, Pending).
2. On-Time Delivery Rate – Measures whether orders were delivered by the expected date.
3. Late Deliveries – Orders that missed the expected delivery date.
4. Average Delivery Time – Time taken from order processing to delivery.
5. Carrier Performance – Effectiveness of shipping companies in meeting deadlines.

In [20]:
df_deliveries.columns

Index(['order_id', 'delivery_status', 'delivery_tracking_id',
       'delivery_shipping_company', 'delivery_expected_delivery_date'],
      dtype='object')

In [21]:
# Convert timestamp 
df_deliveries["delivery_expected_delivery_date"] = pd.to_datetime(df_deliveries["delivery_expected_delivery_date"])

df_deliveries.head()

,order_id,delivery_status,delivery_tracking_id,delivery_shipping_company,delivery_expected_delivery_date
0,ORD1,Delivered,313D0B0E-F,DHL,2025-03-15
1,ORD2,Delivered,6792BDB5-E,BlueDart,2025-03-22
2,ORD3,Shipped,32E8BD2B-E,BlueDart,2025-03-13
3,ORD4,Processing,None,None,2025-03-15
4,ORD5,Processing,None,None,2025-03-18


### **Handling Missing Values**

If tracking_id or shipping_company is missing, they are replaced with "PENDING" and "UNKNOWN", respectively.

In [22]:
# Fill missing values
df_deliveries.fillna({"delivery_tracking_id": "PENDING", 
                    "delivery_shipping_company": "UNKNOWN"}, inplace=True)

df_deliveries.head()

,order_id,delivery_status,delivery_tracking_id,delivery_shipping_company,delivery_expected_delivery_date
0,ORD1,Delivered,313D0B0E-F,DHL,2025-03-15
1,ORD2,Delivered,6792BDB5-E,BlueDart,2025-03-22
2,ORD3,Shipped,32E8BD2B-E,BlueDart,2025-03-13
3,ORD4,Processing,PENDING,UNKNOWN,2025-03-15
4,ORD5,Processing,PENDING,UNKNOWN,2025-03-18


### **Calculating Delivery Status Breakdown**

In [23]:
# Count the number of orders in each status category
delivery_status_counts = df_deliveries["delivery_status"].value_counts()

# Convert the counts into a DataFrame for better readability
df_status_breakdown = pd.DataFrame({"Status": delivery_status_counts.index, 
                                    "Count": delivery_status_counts.values})

# Output Result
print(df_status_breakdown)

       Status   Count
0   Delivered  400190
1     Shipped  200080
2     Pending  199445
3  Processing  100254
4   Cancelled   50047
5  In Transit   49984


### **Calculating On-Time and Delayed Delivery Rate**

In [24]:
# Expected Delivery Dates
df_deliveries.head()

,order_id,delivery_status,delivery_tracking_id,delivery_shipping_company,delivery_expected_delivery_date
0,ORD1,Delivered,313D0B0E-F,DHL,2025-03-15
1,ORD2,Delivered,6792BDB5-E,BlueDart,2025-03-22
2,ORD3,Shipped,32E8BD2B-E,BlueDart,2025-03-13
3,ORD4,Processing,PENDING,UNKNOWN,2025-03-15
4,ORD5,Processing,PENDING,UNKNOWN,2025-03-18


In [25]:
# Actual Delivery Dates
df_order_history.head()

,status,timestamp,order_id,step
0,Processing,2025-03-13 12:13:45,ORD1,1
1,Shipped,2025-03-15 12:13:45,ORD1,2
2,Delivered,2025-03-17 12:13:45,ORD1,3
3,Processing,2025-03-17 16:12:45,ORD2,1
4,Shipped,2025-03-20 16:12:45,ORD2,2


In [26]:
# Extract actual delivery dates
df_actual_delivery = df_order_history[df_order_history["status"] == "Delivered"].copy()
df_actual_delivery = df_actual_delivery.groupby("order_id")["timestamp"].max().reset_index()
df_actual_delivery.rename(columns={"timestamp": "actual_delivery_date"}, inplace=True)

print("Actual Delivery Date:\n", df_actual_delivery)
print()

Actual Delivery Date:
           order_id actual_delivery_date
0             ORD1  2025-03-17 12:13:45
1           ORD100  2025-03-19 16:27:45
2         ORD10000  2025-03-14 18:31:47
3       ORD1000000  2025-03-14 18:35:11
4        ORD100004  2025-03-19 20:34:05
...            ...                  ...
400185   ORD999989  2025-03-13 12:09:11
400186    ORD99999  2025-03-14 22:20:05
400187   ORD999991  2025-03-14 10:37:11
400188   ORD999992  2025-03-17 14:05:11
400189   ORD999998  2025-03-14 20:25:11

[400190 rows x 2 columns]



In [27]:
# Merge with expected delivery dates
df_merged = df_actual_delivery.merge(df_deliveries[["order_id", "delivery_expected_delivery_date"]], on="order_id", how="inner")

# Calculate On-Time Deliveries
df_merged["on_time"] = df_merged["actual_delivery_date"] <= df_merged["delivery_expected_delivery_date"]

df_merged.head()

,order_id,actual_delivery_date,delivery_expected_delivery_date,on_time
0,ORD1,2025-03-17 12:13:45,2025-03-15,False
1,ORD100,2025-03-19 16:27:45,2025-03-20,True
2,ORD10000,2025-03-14 18:31:47,2025-03-13,False
3,ORD1000000,2025-03-14 18:35:11,2025-03-17,True
4,ORD100004,2025-03-19 20:34:05,2025-03-23,True


In [28]:
# Compute On-Time Delivery Rate
on_time_rate = df_merged["on_time"].mean() * 100  # Convert to percentage
delayed_delivery_rate = 100 - on_time_rate

# Output Results
print(f"On-Time Delivery Rate: {on_time_rate:.2f}%")
print(f"Delayed Deliveries Rate: {delayed_delivery_rate:.2f}%")

On-Time Delivery Rate: 50.10%
Delayed Deliveries Rate: 49.90%


### **Calculating Average Delivery Time**

In [29]:
# Get Order Processing Time (earliest Processing timestamp per order)
df_processing = df_order_history[df_order_history["status"] == "Processing"].copy()
df_processing = df_processing.groupby("order_id")["timestamp"].min().reset_index()
df_processing.rename(columns={"timestamp": "order_processing_date"}, inplace=True)

df_processing.head()

,order_id,order_processing_date
0,ORD1,2025-03-13 12:13:45
1,ORD10,2025-03-09 14:35:45
2,ORD100,2025-03-17 16:27:45
3,ORD1000,2025-03-12 20:29:45
4,ORD10000,2025-03-09 18:31:47


In [30]:
# Get Actual Delivery Time (latest Delivered timestamp per order)
df_delivered = df_order_history[df_order_history["status"] == "Delivered"].copy()
df_delivered = df_delivered.groupby("order_id")["timestamp"].max().reset_index()
df_delivered.rename(columns={"timestamp": "actual_delivery_date"}, inplace=True)

df_delivered.head()

,order_id,actual_delivery_date
0,ORD1,2025-03-17 12:13:45
1,ORD100,2025-03-19 16:27:45
2,ORD10000,2025-03-14 18:31:47
3,ORD1000000,2025-03-14 18:35:11
4,ORD100004,2025-03-19 20:34:05


In [31]:
# Merge both DataFrames on order_id
df_merged = df_processing.merge(df_delivered, on="order_id")

# Calculate Delivery Time (in days)
df_merged["delivery_time_days"] = (df_merged["actual_delivery_date"] - df_merged["order_processing_date"]).dt.days

df_merged.head()

,order_id,order_processing_date,actual_delivery_date,delivery_time_days
0,ORD1,2025-03-13 12:13:45,2025-03-17 12:13:45,4
1,ORD100,2025-03-17 16:27:45,2025-03-19 16:27:45,2
2,ORD10000,2025-03-09 18:31:47,2025-03-14 18:31:47,5
3,ORD1000000,2025-03-12 18:35:11,2025-03-14 18:35:11,2
4,ORD100004,2025-03-16 20:34:05,2025-03-19 20:34:05,3


In [32]:
# Compute Average Delivery Time
average_delivery_time = df_merged["delivery_time_days"].mean()

# Output Result
print(f"Average Delivery Time: {average_delivery_time:.2f} days")

Average Delivery Time: 4.00 days


## **Calculating Carrier Performance**

In [33]:
# Merge both dataframes to get actual delivery date
df_merged = df_deliveries.merge(df_order_history, on="order_id", how="left")

df_merged.head()

,order_id,delivery_status,delivery_tracking_id,delivery_shipping_company,delivery_expected_delivery_date,status,timestamp,step
0,ORD1,Delivered,313D0B0E-F,DHL,2025-03-15,Processing,2025-03-13 12:13:45,1
1,ORD1,Delivered,313D0B0E-F,DHL,2025-03-15,Shipped,2025-03-15 12:13:45,2
2,ORD1,Delivered,313D0B0E-F,DHL,2025-03-15,Delivered,2025-03-17 12:13:45,3
3,ORD2,Delivered,6792BDB5-E,BlueDart,2025-03-22,Processing,2025-03-17 16:12:45,1
4,ORD2,Delivered,6792BDB5-E,BlueDart,2025-03-22,Shipped,2025-03-20 16:12:45,2


In [34]:
# Calculate on-time delivery (Actual Date <= Expected Date)
df_merged["on_time"] = df_merged["timestamp"] <= df_merged["delivery_expected_delivery_date"]

df_merged.head()

,order_id,delivery_status,delivery_tracking_id,delivery_shipping_company,delivery_expected_delivery_date,status,timestamp,step,on_time
0,ORD1,Delivered,313D0B0E-F,DHL,2025-03-15,Processing,2025-03-13 12:13:45,1,True
1,ORD1,Delivered,313D0B0E-F,DHL,2025-03-15,Shipped,2025-03-15 12:13:45,2,False
2,ORD1,Delivered,313D0B0E-F,DHL,2025-03-15,Delivered,2025-03-17 12:13:45,3,False
3,ORD2,Delivered,6792BDB5-E,BlueDart,2025-03-22,Processing,2025-03-17 16:12:45,1,True
4,ORD2,Delivered,6792BDB5-E,BlueDart,2025-03-22,Shipped,2025-03-20 16:12:45,2,True


In [36]:
# Calculate carrier performance
carrier_performance = df_merged.groupby("delivery_shipping_company")["on_time"].agg(["sum", "count"])
carrier_performance["on_time_rate"] = (carrier_performance["sum"] / carrier_performance["count"]) * 100

# Format results
carrier_performance = carrier_performance[["on_time_rate"]].reset_index()

# Output Result
print(carrier_performance)

  delivery_shipping_company  on_time_rate
0                  BlueDart     81.816928
1                       DHL     82.073783
2                     FedEx     81.832443
3                   UNKNOWN    100.000000
4                       UPS     81.832748
5                      USPS     81.906864


## **Additional Analysis**

### **Sales & Revenue Analysis**
**Objective:** Identify top-performing products & revenue trends  
**Why?** Helps in stock management, promotions, and pricing strategies  

**Analysis to Perform:**
- Find the best-selling products by quantity & revenue
- Identify underperforming products (low sales, high returns)
- Track monthly/yearly revenue trends
- Measure impact of discounts on sales
- Analyze seasonal trends in purchases

### **Payment Method Preferences**
**Objective:** Understand which payment methods are most used  
**Why?** Helps in improving checkout experience and offering better payment options  

**Analysis to Perform:**
- Find most used payment methods
- Compare discount amounts given per payment method
- Identify high-risk payment methods (refunds, failed transactions)

### **Order Processing & Delivery Efficiency**
**Objective:** Measure how efficient the delivery process is  
**Why?** Helps in improving delivery times, reducing delays, and optimizing logistics  

**Analysis to Perform:**
- Find average time taken for orders to be delivered
- Identify most delayed orders & reasons for delays
- Compare delivery performance across shipping companies
- Track cancelled or returned orders

### **Order History & Status Tracking**
**Objective:** Track the journey of an order from placement to delivery  
**Why?** Helps in improving customer service, resolving complaints, and optimizing order fulfillment

**Analysis to Perform:**
- Track percentage of orders in each stage (Processing, Shipped, Delivered, Cancelled)
- Analyze how long each stage takes on average
- Find patterns in delayed or cancelled orders